<a href="https://colab.research.google.com/github/kushalshah7/Spinach-Disease-Detection/blob/main/IPD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import os
import matplotlib.pyplot as plt
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# import os

# base_dir = '/content/drive/MyDrive/Colab Notebooks/Dataset'
# dataset_dir = os.path.join(base_dir, 'Dataset')
# gan_train_dir = os.path.join(base_dir, 'GAN_Train')
# gan_output_dir = os.path.join(base_dir, 'GAN_Images')

# os.makedirs(gan_train_dir, exist_ok=True)
# os.makedirs(gan_output_dir, exist_ok=True)


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import os
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
z_dim = 100
image_size = 64
batch_size = 64
num_epochs = 20  # Change as needed
learning_rate = 0.0002
beta1 = 0.5

# Transform
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512), nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256), nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)


In [ ]:
def train_gan_on_class(class_name, data_path, save_dir, num_generate=500):
    dataset = ImageFolder(root=data_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    netG = Generator().to(device)
    netD = Discriminator().to(device)

    criterion = nn.BCELoss()
    optimizerD = torch.optim.Adam(netD.parameters(), lr=learning_rate, betas=(beta1, 0.999))
    optimizerG = torch.optim.Adam(netG.parameters(), lr=learning_rate, betas=(beta1, 0.999))

    real_label = 1.
    fake_label = 0.

    print(f"Training GAN for class: {class_name}")
    for epoch in range(num_epochs):
        for i, (real_images, _) in enumerate(dataloader):
            b_size = real_images.size(0)
            real_images = real_images.to(device)
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

            # Train Discriminator
            netD.zero_grad()
            output = netD(real_images)
            errD_real = criterion(output, label)
            errD_real.backward()

            noise = torch.randn(b_size, z_dim, 1, 1, device=device)
            fake = netG(noise)
            label.fill_(fake_label)
            output = netD(fake.detach())
            errD_fake = criterion(output, label)
            errD_fake.backward()
            optimizerD.step()

            # Train Generator
            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward()
            optimizerG.step()

        print(f"Epoch [{epoch+1}/{num_epochs}] Loss_D: {errD_real+errD_fake:.4f}, Loss_G: {errG:.4f}")

        # Show 5 samples at every 20th epoch
        if (epoch + 1) % 20 == 0:
            netG.eval()
            with torch.no_grad():
                z = torch.randn(5, z_dim, 1, 1, device=device)
                samples = netG(z).cpu()
                grid = torchvision.utils.make_grid(samples, nrow=5, normalize=True)
                plt.figure(figsize=(10, 2))
                plt.imshow(grid.permute(1, 2, 0))
                plt.axis("off")
                plt.title(f"{class_name} - Generated samples at epoch {epoch+1}")
                plt.show()
            netG.train()

    # Save generated images
    os.makedirs(save_dir, exist_ok=True)
    netG.eval()
    for i in range(num_generate):
        with torch.no_grad():
            z = torch.randn(1, z_dim, 1, 1, device=device)
            generated = netG(z)
            save_image(generated, f"{save_dir}/{class_name}_gan_{i}.png", normalize=True)


In [ ]:
import shutil

def merge_generated_images(generated_dir, main_dataset_dir):
    for class_name in os.listdir(generated_dir):
        gen_class_path = os.path.join(generated_dir, class_name)
        main_class_path = os.path.join(main_dataset_dir, class_name)

        os.makedirs(main_class_path, exist_ok=True)
        for img_file in os.listdir(gen_class_path):
            src = os.path.join(gen_class_path, img_file)
            dst = os.path.join(main_class_path, img_file)
            shutil.copy(src, dst)
    print("Merged generated images into main dataset.")


In [ ]:
from torchvision.datasets.folder import default_loader
from torch.utils.data import Dataset

# Custom dataset to load images directly from flat class folders
class SimpleImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.file_paths = [
            os.path.join(root_dir, fname) for fname in os.listdir(root_dir)
            if fname.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.transform = transform
        self.loader = default_loader

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = self.loader(self.file_paths[idx])
        if self.transform:
            img = self.transform(img)
        return img, 0  # Dummy label


# Replace ImageFolder with SimpleImageDataset in train_gan_on_class
def train_gan_on_class(class_name, data_path, save_dir, num_generate=500):
    dataset = SimpleImageDataset(data_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # ... (rest of your GAN training code remains unchanged)


In [ ]:
import os

base_path = "/content/drive/My Drive/Colab Notebooks/Dataset"
generated_path = "/content/drive/My Drive/Colab Notebooks/Generated"

# Ensure generated path exists
os.makedirs(generated_path, exist_ok=True)

for class_name in os.listdir(base_path):
    class_dir = os.path.join(base_path, class_name)
    if not os.path.isdir(class_dir):
        continue  # Skip .DS_Store or other files

    # Skip folders like "GAN_Train" or "GAN_Images"
    if class_name.startswith("GAN"):
        print(f"Skipping {class_name} — likely output folders.")
        continue

    # Check for valid images
    valid_imgs = [
        f for f in os.listdir(class_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    if len(valid_imgs) == 0:
        print(f"Skipping {class_name} — no valid images found.")
        continue

    gen_dir = os.path.join(generated_path, class_name)
    train_gan_on_class(class_name, class_dir, gen_dir, num_generate=500)

# Only merge if generated_path has subfolders with images
if os.path.exists(generated_path) and any(os.scandir(generated_path)):
    merge_generated_images(generated_path, base_path)
else:
    print("No generated images found to merge.")



Skipping GAN_Train — likely output folders.
Skipping GAN_Images — likely output folders.
No generated images found to merge.


In [ ]:
# num_generate = 500
# z = torch.randn(num_generate, latent_size, 1, 1).to(device)
# gen_images = G(z)
# for i, img in enumerate(gen_images):
#     save_image(img, f'{gan_output_dir}/gan_img_{i}.png', normalize=True)


In [ ]:
# # import shutil
# # import os

# # for img_file in os.listdir(gan_output_dir):
# #     shutil.copy(os.path.join(gan_output_dir, img_file), os.path.join(dataset_dir, 'Anthracnose'))
# import shutil
# import os

# src_dir = gan_output_dir
# dst_dir = os.path.join(dataset_dir, 'Anthracnose')  # make sure this exists

# os.makedirs(dst_dir, exist_ok=True)  # ensure target folder exists

# for img_file in os.listdir(src_dir):
#     full_src_path = os.path.join(src_dir, img_file)
#     full_dst_path = os.path.join(dst_dir, img_file)

#     if os.path.isfile(full_src_path):  # skip any subdirs, just in case
#         shutil.copy(full_src_path, full_dst_path)



In [ ]:
class SpinachDiseaseModel(nn.Module):
    def __init__(self, num_classes=5):
        super(SpinachDiseaseModel, self).__init__()
        self.model = models.resnet50(pretrained=True)

        for param in self.model.parameters():
            param.requires_grad = False  # Freeze all layers initially

        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    model.to(device)
    train_acc_list, val_acc_list = [], []

    for epoch in range(epochs):
        model.train()
        correct, total, running_loss = 0, 0, 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            running_loss += loss.item()

        train_accuracy = 100 * correct / total
        train_acc_list.append(train_accuracy)

        val_accuracy = evaluate(model, val_loader, device)
        val_acc_list.append(val_accuracy)

        print(f"\nEpoch {epoch+1}: Loss = {running_loss:.4f}, Train Acc = {train_accuracy:.2f}%, Val Acc = {val_accuracy:.2f}%")

    return train_acc_list, val_acc_list

def evaluate(model, data_loader, device):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return 100 * correct / total


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])

train_dataset = ImageFolder(root="dataset/train", transform=transform)
val_dataset = ImageFolder(root="dataset/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpinachDiseaseModel(num_classes=5)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.model.fc.parameters(), lr=1e-4)

train_acc, val_acc = train(model, train_loader, val_loader, criterion, optimizer, epochs=10, device=device)


In [ ]:
from lime.lime_image import LimeImageExplainer
from skimage.segmentation import mark_boundaries
import numpy as np
import matplotlib.pyplot as plt

# Preprocess the image for LIME
def preprocess_image_lime(img_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = Image.open(img_path)
    img = transform(img).unsqueeze(0)
    return img

# Function to predict using the model (LIME needs this)
def predict_fn(images):
    images = torch.tensor(images).float()
    outputs = model(images)
    return F.softmax(outputs, dim=1).detach().numpy()

# LIME explanation
def explain_with_lime(image_path):
    img = preprocess_image_lime(image_path)
    img_numpy = img.squeeze().numpy().transpose(1, 2, 0)  # Convert to HWC format

    explainer = LimeImageExplainer()
    explanation = explainer.explain_instance(img_numpy, predict_fn, top_labels=5, hide_color=0, num_samples=1000)

    # Get the explanation for the top predicted class
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
    plt.imshow(mark_boundaries(temp, mask))
    plt.axis('off')
    plt.show()

# Example usage
explain_with_lime('path_to_image.jpg')
